#Fast Food SVM
Application of the Fast Food kernel expansion algorithm to SVMs.

In [22]:
from __future__ import division, print_function
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
#import pylab as pl
#import theano
#import theano.tensor as T
import numpy as np
from numpy.linalg import norm
#import pandas as pd
import sklearn.datasets
from sklearn import svm
from scipy.linalg import hadamard
from scipy.special import gammaincinv
from functools import partial
from collections import namedtuple

VERBOSE = True  # increases verbosity of some outputs

# random number generator
rng = np.random.RandomState()

# load data
iris = sklearn.datasets.load_iris()
X = iris.data.T # each column is an input pattern
y = iris.target
print('Shape of X:',X.shape,'; shape of y:',y.shape)

# pad with zeros so d is nearest power of 2
# N = X.shape[0]
# l = X.shape[1]
# print('Original shape of X: n =',N,', d =',l)
# d = int(2 ** np.ceil(np.log2(l)))
# if d != l:  # pad only if needed
#     print('Padding input from d =',l,'to d =',d)
#     X = np.pad(X,((0,d-l),(0,0)),mode='constant',constant_values=0)
    
# convert to shared tensors
#X = theano.shared(X,name='X',borrow=True)
#y = theano.shared(y,name='y',borrow=True)

Shape of X: (4, 150) ; shape of y: (150,)


In [23]:
FFPara = namedtuple('FFPara', 'B G PI S')
def fastfood_params(n,d):
    l = int(np.ceil(np.log2(d)))
    d = 2**l
    k = int(np.ceil(n/d))
    n = d*k

    # M = np.diag(v) on each vector v to diagonalize it into the matrix M
    
    
    B = []
    G = []
    PI = []
    S = []
    for ii in xrange(k):
        B_ii  = rng.choice([-1,1],size=k)
        G_ii  = rng.normal(size=k)
        PI_ii = rng.permutation(d)
        
        B.append(B_ii)
        G.append(G_ii)
        PI.append(PI_ii)
        
        p1 = rng.uniform(size=d)
        p2 = d/2
#        print('p1 =',p1,'; p2 =',p2)
        T = gammaincinv(p2,p1)
#        print('T1 =',T)
        T = (T*2) ** (1/2)
#        print('T2 =',T)
        s_i = T * norm(G,'fro')**(-1)
#        print('s_i =', s_i)
        S_ii = s_i
        S.append(S_ii)
    
    S1 = np.zeros(n)
    for ii in xrange(k):
        S1[(ii-1)*d+1:ii+*d] = S[ii]
    
    return FFPara(B, G, PI, S)
print('Ready to generate fastfood params')

SyntaxError: invalid syntax (<ipython-input-23-a72a880dadeb>, line 38)

In [ ]:
def FWHT(X):
    # Fast Walsh-Hadamard Transform for 1D signals
    # of length n=2^M only (non error-proof for now)
    x=get_bit_reversed_list(X)
    x=np.array(x)
    N=len(X)
 
    for i in range(0,N,2):
        x[i]=x[i]+x[i+1]
        x[i+1]=x[i]-2*x[i+1]
 
    L=1
    y=np.zeros_like(x)
    for n in range(2,int(log(N,2))+1):
        M=2**L
        J=0; K=0
        while(K<N):
            for j in range(J,J+M,2):
                y[K]   = x[j]   + x[j+M]
                y[K+1] = x[j]   - x[j+M]
                y[K+2] = x[j+1] + x[j+1+M]
                y[K+3] = x[j+1] - x[j+1+M]
                K=K+4
            J=J+2*M
        x=y.copy()
        L=L+1
 
    y=x/float(N)
    
    return y

def fastfood_forkernel(X,para,sgm,use_spiral=False):
    d0, m = X.shape
    l = int(np.ceil(np.log2(d0)))
    d = 2**l
    if d == d0:
        XX = X
    else:
        XX = np.zeros((d,m))
        XX[1:d0,:] = X
    
    k = len(para.B)
    n = d*k
    tht = np.zeros((n,m))
    for ii in xrange(k):
        B = para.B[ii]
        G = para.G[ii]
        PI = para.PI[ii]
        XX = np.dot(T,G*d)
        tht[(ii-1)*d+1:ii*d,:] = FWHT(T,d)
    S = para.S
    tht = np.dot(tht,S*np.sqrt(d))
    
    T = tht/sgm
    phi = np.concat([np.cos(T),np.sin(T)],axis=1)
    phi = 1.0/np.sqrt(n) * phi
    return phi,tht

In [ ]:
d = 100 # dimension of input pattern
n = d*20 # basis number used for approximation
sgm = 10 # bandwidth for Gaussian kernel

para = fastfood_params(n,d)
#print('Fastfood params:',params)
phi = fastfood_forkernel(X,para,sgm)
print('phi =',phi)